In [1]:
year = 2023
day = 24

In [2]:
%pip install sympy


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [23]:
from aocd import submit
from aocd.models import Puzzle
from functools import reduce
import numpy as np

np.set_printoptions(edgeitems=30, linewidth=100000,
                    formatter=dict(float=lambda x: "%s" % x))

puzzle = Puzzle(year=year, day=day)
data = puzzle.input_data
left_bound = 200_000_000_000_000
right_bound = 400_000_000_000_000
# data = puzzle.examples[0].input_data
# left_bound = 7
# right_bound = 27

data = data.strip()
data = data.split("\n")
data = [list(map(int, d.replace("@", "").replace(",", "").split())) for d in data]
data[0]

[291493672529314, 259618209733833, 379287136024123, -9, 119, -272]

In [27]:
def solve_lines(left, right):
    a, b, c, d = left[3], left[0], right[3], right[0]
    e, f, g, h = left[4], left[1], right[4], right[1]
    try:
        t2 = (a * (h - f) + e * (b - d)) / (c * e - g * a)
        t1 = (t2 * c + d - b) / a
    except ZeroDivisionError:
        # parallel lines
        return None, None, None, None

    xl = t1 * a + b
    xb = t2 * c + d
    yl = t1 * e + f
    yb = t2 * g + h

    return xl, yl, t1, t2

In [28]:
def check(l, r):
    x, y, t1, t2 = solve_lines(l, r)
    if not x:
        return False
    return (left_bound <= x <= right_bound) and (left_bound <= y <= right_bound) and (t1 >= 0) and (t2 >= 0)

In [29]:
from itertools import combinations

answer = sum([check(l, r) for l, r in combinations(data, 2)])
submit(answer, part="a", year=year, day=day)

Part a already solved with same answer: 17776


In [30]:
from sympy import solve, Symbol
from sympy.utilities.lambdify import lambdify

t1 = Symbol("t1")
t2 = Symbol("t2")
a1 = Symbol("a1")
b1 = Symbol("b1")
c1 = Symbol("c1")
d1 = Symbol("d1")
e1 = Symbol("e1")
f1 = Symbol("f1")
a2 = Symbol("a2")
b2 = Symbol("b2")
c2 = Symbol("c2")
d2 = Symbol("d2")
e2 = Symbol("e2")
f2 = Symbol("f2")

alpha = Symbol("alpha")
beta = Symbol("beta")
upsilon = Symbol("upsilon")
delta = Symbol("delta")
gamma = Symbol("gamma")
epsilon = Symbol("epsilon")

sol = solve([a1 * t1 + c1 - (alpha * t1 + upsilon),
             b1 * t1 + d1 - (beta * t1 + delta),
             a2 * t2 + c2 - (alpha * t2 + upsilon),
             b2 * t2 + d2 - (beta * t2 + delta)],
            [t1,
             t2,
             delta,
             upsilon])

compute_delta = lambdify([a1, a2, b1, b2, c1, c2, d1, d2, alpha, beta], sol[delta])
compute_upsilon = lambdify([a1, a2, b1, b2, c1, c2, d1, d2, alpha, beta], sol[upsilon])
compute_t1 = lambdify([a1, a2, b1, b2, c1, c2, d1, d2, alpha, beta], sol[t1])
compute_t2 = lambdify([a1, a2, b1, b2, c1, c2, d1, d2, alpha, beta], sol[t2])

In [31]:
sol2 = solve([a1 * t1 + c1 - (alpha * t1 + upsilon),
             b1 * t1 + d1 - (beta * t1 + delta),
             e1 * t1 + f1 - (gamma * t1 + epsilon),
             a2 * t2 + c2 - (alpha * t2 + upsilon),
             b2 * t2 + d2 - (beta * t2 + delta),
             e2 * t2 + f2 - (gamma * t2 + epsilon)
             ],
            [gamma, epsilon])
compute_gamma = lambdify([e1, f1, e2, f2, t1, t2], sol2[gamma])
compute_epsilon = lambdify([e1, f1, e2, f2, t1, t2], sol2[epsilon])

In [32]:
from itertools import product
SEARCH_RANGE = 300
for alpha, beta in product(range(-SEARCH_RANGE, SEARCH_RANGE), range(-SEARCH_RANGE, SEARCH_RANGE)):
    # for beta in :

    deltas = []
    upsilons = []

    N = 3

    for i in range(N):

        l, r = data[i], data[i + 1]
        a1, b1, c1, d1, e1, f1 = l[3], l[4], l[0], l[1], l[5], l[2]
        a2, b2, c2, d2, e2, f2 = r[3], r[4], r[0], r[1], r[5], r[2]

        try:
            delta = compute_delta(a1, a2, b1, b2, c1, c2, d1, d2, alpha, beta)
            upsilon = compute_upsilon(a1, a2, b1, b2, c1, c2, d1, d2, alpha, beta)
        except ZeroDivisionError:
            continue

        deltas.append(delta)
        upsilons.append(upsilon)

    if np.ptp(deltas) < 0.001 and np.ptp(upsilons) < 0.001 and len(deltas) == N:
        t1 = compute_t1(a1, a2, b1, b2, c1, c2, d1, d2, alpha, beta)
        t2 = compute_t2(a1, a2, b1, b2, c1, c2, d1, d2, alpha, beta)

        gamma = compute_gamma(e1, f1, e2, f2, t1, t2)
        epsilon = compute_epsilon(e1, f1, e2, f2, t1, t2)

        print(f"alpha: {alpha}, beta: {beta} upsilon: {int(upsilon)} delta: {int(delta)} t1: {t1} t2: {t2} gamma: {int(gamma)} epsilon: {int(epsilon)}")

        break
else:
    print(f"No solution found")

answer = int(upsilon + delta + epsilon)

alpha: -193, beta: -230 upsilon: 370994826025810 delta: 410411158485339 t1: 805572857023.0 t2: 694955151546.0 gamma: 218 epsilon: 167572107691063


In [33]:
submit(answer, part="b", year=year, day=day)

Part b already solved with same answer: 948978092202212
